## Exercise: Classifying the MNIST Dataset with Keras
---

In [35]:
# Load MNIST dataset
# 60000 handwritten drawings of digits, uploaded as 28x28 greyscale images 
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Describe the data
img_rows = 28
img_cols = 28
num_classes = 10  # classifying numbers between 0-9

### Data Pipeline
---
Data is modified to a format that can be used to train the model or be evaluated by the model
* Reshape
* Normalize

In [36]:
# Tensorflow by default supports the ndarray format: 
#    NHWC (Number of samples, Height of each sample, Width of each sample, # of channels)
# Need to convert input data from NHW to NHWC, where C=1 because images are greyscale
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)

# Define input shape of a single record
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

### Modeling
___

In [48]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dropout, Softmax
from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D
from tensorflow.keras.optimizers import Adam, SGD

model = Sequential()
model.add(layer=Conv2D(32, kernel_size=(3,3),
                       activation='relu',
                       input_shape=input_shape))
print(model.output_shape)
model.add(layer=Conv2D(32, kernel_size=(3,3),
                       activation='relu'))
print(model.output_shape)
model.add(MaxPooling2D(pool_size=(2,2)))
print(model.output_shape)
model.add(Flatten())
print(model.output_shape)
model.add(Dropout(0.5))
print(model.output_shape)
model.add(Dense(num_classes, activation='softmax'))
print(model.output_shape)

(None, 26, 26, 32)
(None, 24, 24, 32)
(None, 12, 12, 32)
(None, 4608)
(None, 4608)
(None, 10)


In [38]:
x_train[0]

array([[[0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ]],

       [[0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        